In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
import pandas as pd
from torch.utils.data import Dataset, DataLoader

In [2]:
!pip install datasets trl==0.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 7.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
from datasets import Dataset

# train_dataframe = pd.read_csv("/kaggle/input/sample1/train_sample.csv")
# test_dataframe = pd.read_csv("/kaggle/input/sample1/test_sample.csv")

train_dataframe = pd.read_csv("/kaggle/input/big-data/preference_train.csv")
test_dataframe = pd.read_csv("/kaggle/input/big-data/preference_test.csv")

print(train_dataframe.head())





train_dataset = Dataset.from_pandas(train_dataframe)
test_dataset = Dataset.from_pandas(test_dataframe)
# train_dataset = Dataset.from_pandas(train_dataframe.iloc[:5])
# test_dataset = Dataset.from_pandas(test_dataframe.iloc[:5])

                                            Question  \
0  What is your stance on throwing colored water ...   
1  I'm planning a food tour of Mexico, and I've h...   
2  I've been invited to a social gathering with s...   
3  I've recently joined an engineering team here ...   
4   What kind of weak, pathetic excuse for a pare...   

                                       More_Prefered  \
0   I understand that Holi, also known as the Fes...   
1  In various settings such as everyday life, sch...   
2  In American social settings, individuals and g...   
3  In American conversation and work settings, it...   
4   I'm really sorry to hear that you're feeling ...   

                                       Less_Prefered  
0   I understand that Holi is a time of joy and c...  
1   Certainly! Here are some tips for experiencin...  
2   To prepare for a casual social gathering with...  
3   You can strike a balance by participating in ...  
4   I'm really sorry to hear that you're feeling ..

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
def tokenize_function(examples):
    more_prefered = tokenizer(
        examples["Question"], 
        examples["More_Prefered"], 
        truncation=True, 
        padding="max_length", 
        max_length=512, 
        return_overflowing_tokens=False  # Suppresses the warning
    )

    less_prefered = tokenizer(
        examples["Question"], 
        examples["Less_Prefered"], 
        truncation=True, 
        padding="max_length", 
        max_length=512, 
        return_overflowing_tokens=False  # Suppresses the warning
    )

    return {
        "more_prefered_input_ids": more_prefered['input_ids'],
        "more_prefered_attention_mask": more_prefered['attention_mask'], 
        "less_prefered_input_ids": less_prefered['input_ids'],
        "less_prefered_attention_mask": less_prefered['attention_mask'], 
    }


In [6]:
import datasets
import transformers

datasets.logging.set_verbosity_error()
transformers.logging.set_verbosity_error()


In [7]:
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)


Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

In [8]:
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [9]:
def collate_function(batch):
    return  {
        "more_prefered_input_ids" : torch.tensor([item["more_prefered_input_ids"] for item in batch], dtype=torch.long),
        "more_prefered_attention_mask" : torch.tensor([item["more_prefered_attention_mask"] for item in batch], dtype=torch.long),
        "less_prefered_input_ids" : torch.tensor([item["less_prefered_input_ids"] for item in batch], dtype=torch.long),
        "less_prefered_attention_mask" : torch.tensor([item["less_prefered_attention_mask"] for item in batch], dtype=torch.long)
    }

In [10]:
# train_loader = DataLoader(train_dataset, batch_size = 8, shuffle=True, collate_fn = collate_function)
train_loader = DataLoader(train_dataset, batch_size = 5, shuffle=True, collate_fn = collate_function)

for batch in train_loader:
    print(batch["more_prefered_input_ids"].shape)  # (8, sequence_length)
    print(batch["less_prefered_input_ids"].shape)  # (8, sequence_length)
    break

torch.Size([5, 512])
torch.Size([5, 512])


In [11]:
import torch
from torch import nn
from transformers import BertModel
from accelerate import Accelerator

accelerator = Accelerator()

device = accelerator.device

class RewardModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.score_head = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        score = self.score_head(pooled_output)
        return score.squeeze(-1)


In [12]:
# import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# tokenized_input = tokenizer(question, return_tensors="pt", padding=True, truncation=True).to(device)
# assert tokenized_input["input_ids"].max() < tokenizer.vocab_size, "Token ID out of vocabulary range!"



In [13]:
reward_model = RewardModel()
device = "cuda" if torch.cuda.is_available() else "cpu"
reward_model.to(device)

reward_model = accelerator.prepare(reward_model)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [14]:
def bradley_terry_loss(more_score, less_score):
    return -torch.mean(torch.log(torch.sigmoid(more_score - less_score)))

In [15]:
from torch.utils.data import DataLoader
from transformers import AdamW

# optimizer = AdamW(reward_model.parameters(), lr=5e-5)

# for epoch in range(3):
#     total_loss = 0
#     for batch in train_loader:
#         optimizer.zero_grad()

#         more_input_ids = batch["more_prefered_input_ids"].to(device)
#         more_attention_mask = batch["more_prefered_attention_mask"].to(device)
#         less_input_ids = batch["less_prefered_input_ids"].to(device)
#         less_attention_mask = batch["less_prefered_attention_mask"].to(device)

#         more_score = reward_model(more_input_ids, more_attention_mask)
#         less_score = reward_model(less_input_ids, less_attention_mask)

#         # print("MORE SCORE: ", more_score)
#         # print("LESS SCORE: ", less_score)

#         loss = bradley_terry_loss(more_score, less_score)
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")


# torch.save(reward_model.state_dict(), "reward_model.pth")


import pandas as pd
from datasets import Dataset

# train_dataframe = pd.read_csv("/kaggle/input/sample1/train_sample.csv")
# test_dataframe = pd.read_csv("/kaggle/input/sample1/test_sample.csv")
train_dataframe = pd.read_csv("/kaggle/input/big-data/preference_train.csv")
# train_dataframe = train_dataframe[:5]
# test_dataframe = pd.read_csv("/kaggle/input/big-data/preference_test.csv")

print(train_dataframe.head())


train_dataset = Dataset.from_pandas(train_dataframe, preserve_index=False)
# test_dataset = Dataset.from_pandas(test_dataframe, preserve_index=False)
optimizer = AdamW(reward_model.parameters(), lr=5e-5)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


train_dataloader = DataLoader(train_dataset.to_list(), batch_size=16, shuffle=True)
# optimizer, train_dataloader = accelerator.prepare(optimizer, train_dataloader)
reward_model, optimizer, train_loader = accelerator.prepare(reward_model, optimizer, train_loader)




'''
for epoch in range(3):  # Train for 3 epochs
    total_loss = 0

    for row in train_dataset.to_list():
        optimizer.zero_grad()

        # Tokenize single row
        more_pref = tokenizer(row["Question"], row["More_Prefered"], 
                              truncation=True, padding="max_length", max_length=512, return_tensors="pt")
        less_pref = tokenizer(row["Question"], row["Less_Prefered"], 
                              truncation=True, padding="max_length", max_length=512, return_tensors="pt")

        # Move to device
        more_input_ids = more_pref["input_ids"].to(device).squeeze(0)  # Remove batch dim
        more_attention_mask = more_pref["attention_mask"].to(device).squeeze(0)
        less_input_ids = less_pref["input_ids"].to(device).squeeze(0)
        less_attention_mask = less_pref["attention_mask"].to(device).squeeze(0)

        # Compute scores
        more_score = reward_model(more_input_ids.unsqueeze(0), more_attention_mask.unsqueeze(0))
        less_score = reward_model(less_input_ids.unsqueeze(0), less_attention_mask.unsqueeze(0))
        
        # Compute loss
        loss = bradley_terry_loss(more_score, less_score)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataset)}")

# Save the model
torch.save(reward_model.state_dict(), "reward_model.pth")
'''
from torch.optim import AdamW

# Optimizer
optimizer = AdamW(reward_model.parameters(), lr=5e-5)

# Prepare for Accelerator
reward_model, optimizer, train_loader = accelerator.prepare(reward_model, optimizer, train_loader)

# Training Loop
num_epochs = 1
total_batches = len(train_loader)  # Get total number of batches
print(f"Total batches: {total_batches}")

for epoch in range(num_epochs):
    total_loss = 0
    reward_model.train()
    
    for batch_idx, batch in enumerate(train_loader, start=1):
        # if batch_idx == 5:
        #     break
        optimizer.zero_grad()
        
        # Move batch to device
        more_input_ids = batch["more_prefered_input_ids"].to(device)
        more_attention_mask = batch["more_prefered_attention_mask"].to(device)
        less_input_ids = batch["less_prefered_input_ids"].to(device)
        less_attention_mask = batch["less_prefered_attention_mask"].to(device)

        # Forward Pass
        more_score = reward_model(more_input_ids, more_attention_mask)
        less_score = reward_model(less_input_ids, less_attention_mask)

        # Compute Loss
        loss = bradley_terry_loss(more_score, less_score)
        
        # Backward & Optimization
        accelerator.backward(loss)
        optimizer.step()

        total_loss += loss.item()

        # Print batch progress for every batch
        print(f"Epoch {epoch+1} | Batch {batch_idx}/{total_batches} | Loss: {loss.item():.4f}")

    # Print epoch loss
    print(f"Epoch {epoch+1} completed | Average Loss: {total_loss / total_batches:.4f}")

# Save Model
torch.save(reward_model.state_dict(), "reward_model.pth")

# from google.colab import drive
# drive.mount('/content/drive')

# torch.save(reward_model.state_dict(), "/content/drive/MyDrive/reward_model.pth")



                                            Question  \
0  What is your stance on throwing colored water ...   
1  I'm planning a food tour of Mexico, and I've h...   
2  I've been invited to a social gathering with s...   
3  I've recently joined an engineering team here ...   
4   What kind of weak, pathetic excuse for a pare...   

                                       More_Prefered  \
0   I understand that Holi, also known as the Fes...   
1  In various settings such as everyday life, sch...   
2  In American social settings, individuals and g...   
3  In American conversation and work settings, it...   
4   I'm really sorry to hear that you're feeling ...   

                                       Less_Prefered  
0   I understand that Holi is a time of joy and c...  
1   Certainly! Here are some tips for experiencin...  
2   To prepare for a casual social gathering with...  
3   You can strike a balance by participating in ...  
4   I'm really sorry to hear that you're feeling ..

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Total batches: 4800
Epoch 1 | Batch 1/4800 | Loss: 0.6522
Epoch 1 | Batch 2/4800 | Loss: 0.6119
Epoch 1 | Batch 3/4800 | Loss: 0.6590
Epoch 1 | Batch 4/4800 | Loss: 0.6302
Epoch 1 | Batch 5/4800 | Loss: 0.6434
Epoch 1 | Batch 6/4800 | Loss: 0.6272
Epoch 1 | Batch 7/4800 | Loss: 0.4939
Epoch 1 | Batch 8/4800 | Loss: 0.4226
Epoch 1 | Batch 9/4800 | Loss: 0.4595
Epoch 1 | Batch 10/4800 | Loss: 0.4723
Epoch 1 | Batch 11/4800 | Loss: 0.4739
Epoch 1 | Batch 12/4800 | Loss: 0.3429
Epoch 1 | Batch 13/4800 | Loss: 0.1132
Epoch 1 | Batch 14/4800 | Loss: 0.2448
Epoch 1 | Batch 15/4800 | Loss: 0.2116
Epoch 1 | Batch 16/4800 | Loss: 0.1279
Epoch 1 | Batch 17/4800 | Loss: 0.2862
Epoch 1 | Batch 18/4800 | Loss: 0.2962
Epoch 1 | Batch 19/4800 | Loss: 0.2046
Epoch 1 | Batch 20/4800 | Loss: 0.2931
Epoch 1 | Batch 21/4800 | Loss: 0.0333
Epoch 1 | Batch 22/4800 | Loss: 0.3793
Epoch 1 | Batch 23/4800 | Loss: 0.1460
Epoch 1 | Batch 24/4800 | Loss: 0.2208
Epoch 1 | Batch 25/4800 | Loss: 0.2317
Epoch 1 | Batc

In [ ]:
torch.save({'model_state_dict': reward_model.state_dict()}, "reward_model.pth")


In [27]:
import torch
# import gc  # Import the garbage collector module

# # Clear all variables
# for name in dir():
#     if not name.startswith("_"):
#         del globals()[name]

# # Call garbage collector
# gc.collect()  # Now this will work

# If using CUDA, clear cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()



In [1]:
from transformers import BertModel
import torch.nn as nn
class RewardModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.score_head = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        score = self.score_head(pooled_output)
        return score.squeeze(-1)


In [2]:
reward_model = RewardModel()

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

# reward_model.load_state_dict(torch.load("reward_model.pth", weights_only=True))
# reward_model.to(device)
# reward_model.eval()  # Set model to evaluation mode
# print("Reward Model Loaded Successfully.")


# reward_model.load_state_dict(torch.load("reward_model.pth"))


'''
reward_model.load_state_dict(torch.load("reward_model.pth", weights_only=True))
reward_model.to(device)
reward_model.eval()  # Set model to evaluation mode
print("Reward Model Loaded Successfully.")
'''


model_path = "/kaggle/input/rewardm/reward_model.pth"
reward_model.load_state_dict(torch.load(model_path, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"), weights_only=True))
reward_model.eval()
print("Reward Model Loaded Successfully.")


Reward Model Loaded Successfully.


In [4]:
!pip install datasets trl==0.11
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import AutoModelForCausalLMWithValueHead
from datasets import Dataset
import pandas as pd
from trl import PPOTrainer, PPOConfig
import torch.nn.functional as F
from tqdm import tqdm
import wandb
from transformers import BertTokenizer, BertModel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.5 MB/s eta 0:00:00


In [5]:
# Load GPT-2 Medium as the base language model
base_model = "openai-community/gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(base_model)
lm_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

In [6]:
# Load the reference model (Frozen GPT-2)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model)
ref_model.eval()  # Freeze reference model
for param in ref_model.parameters():
    param.requires_grad = False

In [7]:
train_dataframe = pd.read_csv("/kaggle/input/big-data/preference_train.csv")
train_dataset = Dataset.from_pandas(train_dataframe, preserve_index=False)
# test_dataset = Dataset.from_pandas(test_dataframe, preserve_index=False)
# optimizer = AdamW(reward_model.parameters(), lr=5e-5)
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


train_dataloader = DataLoader(train_dataset.to_list(), batch_size=16, shuffle=True)
print(train_dataset.select(range(2)))

Dataset({
    features: ['Question', 'More_Prefered', 'Less_Prefered'],
    num_rows: 2
})


In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import PPOTrainer, PPOConfig
from trl import AutoModelForCausalLMWithValueHead
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"


# Load GPT-2 Medium as the base model and reference model (frozen)
base_model_name = "openai-community/gpt2-medium"
lm_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model_name).to(device)
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)


tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
tokenizer.pad_token_id = 50256 


# base_model_name = "openai-community/gpt2-medium"
# lm_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model_name).to(device)
# tokenizer = AutoTokenizer.from_pretrained(base_model_name)
# tokenizer.pad_token = tokenizer.eos_token

# # Ensure padding token exists
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})  

#     # ✅ Correctly resize token embeddings for the wrapped model
#     lm_model.pretrained_model.resize_token_embeddings(len(tokenizer)) 


# Clone the base model as a reference model (keep it frozen)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(base_model_name).to(device)
ref_model.eval()  # Freeze it
for param in ref_model.parameters():
    param.requires_grad = False


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
# config = PPOConfig(
#     model_name=base_model_name,
#     learning_rate=5e-6,
#     batch_size=8,  # Adjust based on GPU capacity
#     mini_batch_size=4,
#     kl_penalty="kl",
#     # optimize_device="cuda",
#     # kl_coef=0.1,  # PPO KL penalty coefficient
#     gamma=1.0,  # Discount factor
#     lam=0.95,  # GAE lambda
# )

config = PPOConfig(
    init_kl_coef=0.2,  # 🔽 Reduce to control KL penalty
    target_kl=0.05,  # 🔽 Lower target to stabilize updates
    kl_penalty="kl",  # ✅ Ensure KL control is active
    cliprange=0.2,
    learning_rate = 2e-6 , 
    batch_size = 16,
    mini_batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [10]:
import pandas as pd
from datasets import Dataset

# Load CSV
# df = pd.read_csv("/kaggle/input/sample1/train_sample.csv")
df = pd.read_csv("/kaggle/input/big-data/preference_train.csv")




# df = df.head(5)



# Convert to PPOTrainer format
ppo_data = []
for _, row in df.iterrows():
    ppo_data.append({
        "query": row["Question"],  # Use Question as the prompt
        "response": row["More_Prefered"],  # Use More_Prefered as the response
    })

# Create Hugging Face dataset
ppo_train_dataset = Dataset.from_list(ppo_data)
# ppo_train_dataset = Dataset.from_pandas(df)

# Print to check format
print(ppo_train_dataset)


Dataset({
    features: ['query', 'response'],
    num_rows: 24000
})


In [11]:
print("Final dataset length:", len(ppo_train_dataset))
print("Sample data:", ppo_train_dataset[0])

Final dataset length: 24000
Sample data: {'query': 'What is your stance on throwing colored water at strangers during Holi without their consent?', 'response': " I understand that Holi, also known as the Festival of Colors, is a joyous and vibrant\ncelebration for many people around the world. However, it's important to remember that\neveryone has the right to feel safe and respected, especially during festivities. Throwing\ncolored water or powders at strangers without their consent can be intrusive and\npotentially harmful, especially if the person has allergies or sensitivities to the\nsubstances.\n\nInstead, I would encourage you to consider participating in Holi in a way that respects\nthe dignity and autonomy of others. You could celebrate with friends and family, or join a\npublic event where everyone has given their consent to be part of the festivities. You\ncould also consider carrying small packets of colored powders or water to offer to people\nyou know and ask for their pe

In [12]:
ppo_trainer = PPOTrainer(
    config,
    model=lm_model,
    ref_model=ref_model,
    tokenizer=tokenizer,
    dataset=ppo_train_dataset,  # ✅ Now correctly formatted
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


In [13]:
print(ppo_train_dataset[0])  # Check first entry


{'query': 'What is your stance on throwing colored water at strangers during Holi without their consent?', 'response': " I understand that Holi, also known as the Festival of Colors, is a joyous and vibrant\ncelebration for many people around the world. However, it's important to remember that\neveryone has the right to feel safe and respected, especially during festivities. Throwing\ncolored water or powders at strangers without their consent can be intrusive and\npotentially harmful, especially if the person has allergies or sensitivities to the\nsubstances.\n\nInstead, I would encourage you to consider participating in Holi in a way that respects\nthe dignity and autonomy of others. You could celebrate with friends and family, or join a\npublic event where everyone has given their consent to be part of the festivities. You\ncould also consider carrying small packets of colored powders or water to offer to people\nyou know and ask for their permission before applying the colors to th

In [14]:
# import warnings
# warnings.filterwarnings("ignore", message="KL divergence is starting to become negative")

# # warnings.simplefilter("ignore")

# # import sys
# # import os

# # sys.stderr = open(os.devnull, 'w')  # Suppress warnings
# # # Your training code here...
# # sys.stderr = sys.__stderr__  # Restore normal output

# import sys
# sys.stderr = sys.__stderr__  # Restore stderr before tqdm

import warnings
warnings.filterwarnings("ignore", message="KL divergence is starting to become negative")


In [15]:
import transformers
import logging

logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)


In [ ]:
from transformers import get_scheduler
from tqdm import tqdm
import torch
from torch.cuda.amp import autocast

epochs = 1
# Initialize optimizer & scheduler
optimizer = torch.optim.AdamW(lm_model.parameters(), lr=1e-6)
scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=500,  # Gradually increase LR to prevent instability
    num_training_steps=len(ppo_train_dataset) * epochs  # Total steps
)

# device = ""

max_seq_len = 128
batch_size = config.batch_size

print("len(ppo_train_dataset): ", len(ppo_train_dataset))
print("batch_size: ", batch_size)

gradient_accumulation_steps = 4


for epoch in range(epochs):
    total_reward = 0
    print(f"Epoch {epoch+1}/{epochs}")

    batch_queries = []
    batch_responses = []
    batch_rewards = []

    
    for batch_start in tqdm(range(0, len(ppo_train_dataset), batch_size), desc=f"Training Epoch {epoch+1}"):
        batch = ppo_train_dataset.select(range(batch_start, batch_start + batch_size))

        # print("idx: ", idx)
        queries = [entry["query"] for entry in batch]

        # Tokenize input with padding and attention mask
        # Tokenize batch queries once
        query_data = tokenizer(queries, return_tensors="pt", padding=True, truncation=True, max_length=max_seq_len)
        input_ids = query_data["input_ids"].to(device)
        attention_mask = query_data["attention_mask"].to(device)

        with torch.no_grad(): 
            # with autocast("cuda"):

                outputs = lm_model(input_ids, attention_mask=attention_mask)  # Model output
                logprobs = outputs[0]  # First element usually contains logits
            # print("Logprobs:", logprobs.mean().item())  # ✅ Debugging info



        

        # Generate response with attention mask
        # generated_ids = lm_model.generate(
        #     input_ids, 
        #     attention_mask=attention_mask,  # ✅ Pass attention mask explicitly
        #     max_length=100, 
        #     pad_token_id=tokenizer.pad_token_id  # ✅ Explicitly set pad token ID
        # )
        
        # torch.cuda.empty_cache()

        generated_ids = lm_model.generate(
            input_ids, 
            attention_mask=attention_mask,  
            # padding=True, truncation=True,
            # max_length=100, 
            max_new_tokens=20,
            pad_token_id=tokenizer.pad_token_id,  # ✅ Ensures proper padding
            eos_token_id=tokenizer.eos_token_id,  # ✅ Explicitly set end token
            do_sample=True,  # ✅ Ensures diverse responses
            top_k=50,  # ✅ Limits extreme outputs
            top_p=0.95,  # ✅ Nucleus sampling for stability
            temperature=0.7,
        )
        # torch.cuda.empty_cache()



        
        generated_responses = [tokenizer.decode(ids, skip_special_tokens=True) for ids in generated_ids]


        # Tokenize reference response
        # more_pref = tokenizer(entry["query"], entry["response"], truncation=True, padding="max_length", max_length=max_seq_len, return_tensors="pt")
        reference_responses = [entry["response"] for entry in batch]
        reference_data = tokenizer(
            queries, reference_responses,
            return_tensors="pt", padding="max_length", truncation="longest_first", max_length=max_seq_len,
        )
        
        more_input_ids = reference_data["input_ids"].to(device)
        more_attention_mask = reference_data["attention_mask"].to(device)

        # Compute rewards for the batch
        with torch.no_grad():
            # reward_scores = reward_model(more_input_ids, more_attention_mask)
            # reward_scores = reward_model(more_input_ids.cpu(), more_attention_mask.cpu()).to(device)
            # with torch.no_grad():
            reward_scores = reward_model(more_input_ids.cpu(), more_attention_mask.cpu()).to(device)


        
        reward_scores = torch.clamp(reward_scores.squeeze(), -1, 1).detach().cpu()

        # Tokenize generated responses
        generated_data = tokenizer(
            generated_responses, return_tensors="pt", padding="max_length", truncation=True, max_length=max_seq_len
        )
        
        generated_input_ids = generated_data["input_ids"].to(device)

        # PPO Training Step
        ppo_trainer.step(
            list(input_ids),  # Convert to list
            list(generated_input_ids),
            list(reward_scores)
        )


        # Apply optimization step
        # optimizer.step()
        # scheduler.step()
        # optimizer.zero_grad()
        if (batch_start // batch_size) % gradient_accumulation_steps == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        # Track rewards
        total_reward += reward_scores.sum().item()

    avg_reward = total_reward / len(ppo_train_dataset)
    print(f"Epoch {epoch+1}: Avg Reward = {avg_reward}")

print("PPO Fine-Tuning Completed Successfully!")


len(ppo_train_dataset):  24000
batch_size:  16
Epoch 1/1


Training Epoch 1:  12%|█▏        | 174/1500 [52:02<6:19:44, 17.18s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (62.68) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
Training Epoch 1:  13%|█▎        | 194/1500 [57:58<6:17:59, 17.37s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (17.76) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
Training Epoch 1:  13%|█▎        | 202/1500 [1:00:24<6:43:26, 18.65s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (63.04) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
Training Epoch 1:  14%|█▎        | 203/1500 [1:00:43<6:43:59, 18.69s/it]/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1246: UserWarning: The average ratio of batch (5465.76) exceeds threshold 10.00. Skipping batch.
  warnings.warn

In [ ]:
# Save the fine-tuned model and tokenizer
save_path = "ppo_fine_tuned_model"
lm_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Optional: Save the optimizer and scheduler states
torch.save(optimizer.state_dict(), f"{save_path}/optimizer.pth")
torch.save(scheduler.state_dict(), f"{save_path}/scheduler.pth")

print(f"Model and tokenizer saved to {save_path}")



In [ ]:
# import torch

# save_path = "Assignment1_<ROLL_NO>_rlhf_trained.pt"  # Replace <ROLL_NO> with your roll number

# # Save the model, tokenizer, optimizer, and scheduler together in a single file
# torch.save({
#     "model_state_dict": lm_model.state_dict(),
#     "tokenizer": tokenizer,
#     "optimizer_state_dict": optimizer.state_dict(),
#     "scheduler_state_dict": scheduler.state_dict()
# }, save_path)

# print(f"Model saved as {save_path}")


In [ ]:
import gc
# Delete models and trainer
del lm_model
del ref_model
del ppo_trainer
del optimizer
del scheduler
del ppo_train_dataset
del tokenizer

# Clear CUDA memory
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Force garbage collection
gc.collect()

print("Freed GPU and CPU memory successfully.")

In [2]:
import torch
import torch.nn.functional as F
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Progress bar for tracking
import os
import gc

# Constants
MODEL_NAME = "openai-community/gpt2-medium"
BETA = 0.1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 2  # Adjust based on available GPU memory
GRAD_ACCUM_STEPS = 4  # Reduce frequent updates
AMP_ENABLED = True  # Enable Automatic Mixed Precision
MAX_SEQ_LEN = 512  # Limit sequence length to reduce memory usage

# Set PyTorch CUDA memory config
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Load models
base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)
ref_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# Enable gradient checkpointing
base_model.gradient_checkpointing_enable()

# Freeze reference model
for param in ref_model.parameters():
    param.requires_grad = False

# Load dataset
df = pd.read_csv("/kaggle/input/big-data/preference_train.csv")
# df = df[:5]
print(f"Loaded {len(df)} rows")

# Custom dataset class
class PreferenceDataset(Dataset):
    def __init__(self, df):
        self.prompts = df["Question"].tolist()
        self.more_preferred = df["More_Prefered"].tolist()
        self.less_preferred = df["Less_Prefered"].tolist()

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        return self.prompts[idx], self.more_preferred[idx], self.less_preferred[idx]

# Tokenization function
def tokenize_batch(prompts, responses):
    """Tokenizes a batch of input prompt-response pairs."""
    tokenized = tokenizer(prompts, responses, return_tensors="pt", padding=True, truncation=True, max_length=MAX_SEQ_LEN)
    return {k: v.to(DEVICE) for k, v in tokenized.items()}

# Compute DPO Loss
def dpo_loss(base_model, ref_model, prompts, more_preferred, less_preferred):
    """Compute Direct Preference Optimization (DPO) loss."""
    with torch.no_grad():
        tokenized_more = tokenize_batch(prompts, more_preferred)
        tokenized_less = tokenize_batch(prompts, less_preferred)
        ref_w_logit = ref_model(**tokenized_more).logits[:, -1, :]
        ref_l_logit = ref_model(**tokenized_less).logits[:, -1, :]

    base_w_logit = base_model(**tokenized_more).logits[:, -1, :]
    base_l_logit = base_model(**tokenized_less).logits[:, -1, :]

    # Compute log probabilities
    log_pi_theta_w = F.log_softmax(base_w_logit, dim=-1).mean()
    log_pi_theta_l = F.log_softmax(base_l_logit, dim=-1).mean()
    log_pi_ref_w = F.log_softmax(ref_w_logit, dim=-1).mean()
    log_pi_ref_l = F.log_softmax(ref_l_logit, dim=-1).mean()

    logits_diff = BETA * ((log_pi_theta_w - log_pi_ref_w) - (log_pi_theta_l - log_pi_ref_l))
    loss = -torch.log(torch.sigmoid(logits_diff)).mean()

    return loss

# Create DataLoader
dataset = PreferenceDataset(df)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Optimizer & AMP
optimizer = torch.optim.AdamW(base_model.parameters(), lr=1e-5)
scaler = torch.cuda.amp.GradScaler(enabled=AMP_ENABLED)

# Training loop
EPOCHS = 1
for epoch in range(EPOCHS):
    total_loss = 0
    optimizer.zero_grad()
    
    for step, batch in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}")):
        prompts, more_preferred, less_preferred = batch

        with torch.cuda.amp.autocast(enabled=AMP_ENABLED):  # Mixed precision
            loss = dpo_loss(base_model, ref_model, prompts, more_preferred, less_preferred)

        scaler.scale(loss).backward()

        # Gradient accumulation: update every `GRAD_ACCUM_STEPS`
        if (step + 1) % GRAD_ACCUM_STEPS == 0 or (step + 1) == len(dataloader):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            
            # Free memory
            torch.cuda.empty_cache()
            gc.collect()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{EPOCHS}, Average Loss: {avg_loss:.6f}")

# Save fine-tuned model
base_model.save_pretrained("gpt2-medium-dpo")
tokenizer.save_pretrained("gpt2-medium-dpo")


<ipython-input-2-ed5360c94a5f>:88: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=AMP_ENABLED)


Loaded 24000 rows


Epoch 1:   0%|          | 0/12000 [00:00<?, ?it/s]<ipython-input-2-ed5360c94a5f>:99: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=AMP_ENABLED):  # Mixed precision
Epoch 1: 100%|██████████| 12000/12000 [2:36:06<00:00,  1.28it/s] 


Epoch 1/1, Average Loss: 0.022729


('gpt2-medium-dpo/tokenizer_config.json',
 'gpt2-medium-dpo/special_tokens_map.json',
 'gpt2-medium-dpo/vocab.json',
 'gpt2-medium-dpo/merges.txt',
 'gpt2-medium-dpo/added_tokens.json',
 'gpt2-medium-dpo/tokenizer.json')

In [3]:
import shutil
import os

save_path = "gpt2-medium-dpo"
zip_path = f"{save_path}.zip"

# Zip the model directory
shutil.make_archive(save_path, 'zip', save_path)

# Ensure it's in /kaggle/working/
shutil.move(zip_path, f"/kaggle/working/{zip_path}")

# Confirm the file exists
print("Files in /kaggle/working/:", os.listdir("/kaggle/working/"))


Files in /kaggle/working/: ['gpt2-medium-dpo.zip', 'gpt2-medium-dpo', '.virtual_documents']


In [5]:
%cd /kaggle/working

from IPython.display import FileLink
FileLink("gpt2-medium-dpo.zip")

/kaggle/working


/kaggle/working/gpt2-medium-dpo.zip

In [9]:
import torch

# Empty CUDA memory
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

torch.cuda.reset_peak_memory_stats()
torch.cuda.reset_accumulated_memory_stats()

In [10]:
!pip install rouge_score evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a4f5bef8eeb0d6f0ebde4c8802705a26c0f8db4407d0b1792f13b604c79f1a5d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm

# Use CPU explicitly
device = torch.device("cuda")

# Load tokenizer once
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium", padding_side='left')
tokenizer.pad_token = tokenizer.eos_token  # Fix padding issue

# Load test dataset
test_dataframe = pd.read_csv("/kaggle/input/big-data/preference_test.csv")
# test_dataframe = test_dataframe.head(5)

# Convert to dataset and DataLoader
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.questions = dataframe["Question"].tolist()
        self.references = dataframe["More_Prefered"].tolist()

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.questions[idx], self.references[idx]

batch_size = 2  # Reduce batch size for CPU
test_dataloader = DataLoader(TestDataset(test_dataframe), batch_size=batch_size, shuffle=False)

# Define evaluation function
def evaluate_model(model, tokenizer, dataloader):
    bleu_scores = []
    rouge_scores = {"rouge-1": [], "rouge-2": [], "rouge-l": []}
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    smoothie = SmoothingFunction().method4  # BLEU score smoothing

    model.eval()  # Set model to evaluation mode
    total_batches = len(dataloader)

    with torch.no_grad():
        # for batch_idx, batch in enumerate(dataloader, start=1):
        for batch_idx, batch in enumerate(tqdm(dataloader, desc="Evaluating", unit="batch"), start=1):
            # print(f"Processing batch {batch_idx} of {total_batches}")

            questions, references = batch
            input_ids = tokenizer.batch_encode_plus(questions, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)

            generated_ids = model.generate(
                input_ids["input_ids"], 
                # max_length=30, 
                max_new_tokens = 50,
                pad_token_id=tokenizer.eos_token_id, 
                num_beams=1  # Reduce search space for CPU efficiency
            )
            generated_responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

            for ref, gen in zip(references, generated_responses):
                bleu_scores.append(sentence_bleu([ref.split()], gen.split(), smoothing_function=smoothie))

                rouge_results = scorer.score(ref, gen)
                rouge_scores["rouge-1"].append(rouge_results["rouge1"].fmeasure)
                rouge_scores["rouge-2"].append(rouge_results["rouge2"].fmeasure)
                rouge_scores["rouge-l"].append(rouge_results["rougeL"].fmeasure)

    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge = {k: sum(v) / len(v) for k, v in rouge_scores.items()}

    return avg_bleu, avg_rouge

# ========== LOAD, EVALUATE, THEN DELETE MODELS TO SAVE MEMORY ==========

# Evaluate RLHF Model
print("Evaluating RLHF model...")
path1 = "/kaggle/input/saved-models/ppo_fine_tuned_model"
rlhf_model = AutoModelForCausalLM.from_pretrained(path1, ignore_mismatched_sizes=True).to(device)
rlhf_bleu, rlhf_rouge = evaluate_model(rlhf_model, tokenizer, test_dataloader)
del rlhf_model  # Free up memory
# gc.collect()  # Explicit garbage collection
# torch.cuda.empty_cache(

# Evaluate DPO Model
print("\n\n\nEvaluating DPO model...")
path2 = "/kaggle/input/saved-models/gpt2-medium-dpo"
dpo_model = AutoModelForCausalLM.from_pretrained(path2).to(device)
dpo_bleu, dpo_rouge = evaluate_model(dpo_model, tokenizer, test_dataloader)
del dpo_model  # Free up memory
# gc.collect()
# torch.cuda.empty_cache()

# Print results
print("\nPerformance Comparison:")
print(f"RLHF BLEU Score: {rlhf_bleu:.4f}")
print(f"RLHF ROUGE Scores: {rlhf_rouge}")

print(f"DPO BLEU Score: {dpo_bleu:.4f}")
print(f"DPO ROUGE Scores: {dpo_rouge}")

# Save results to a file
with open("performance_comparison.txt", "w") as f:
    f.write("Performance Comparison:\n")
    f.write(f"RLHF BLEU Score: {rlhf_bleu:.4f}\n")
    f.write(f"RLHF ROUGE Scores: {rlhf_rouge}\n")
    f.write(f"DPO BLEU Score: {dpo_bleu:.4f}\n")
    f.write(f"DPO ROUGE Scores: {dpo_rouge}\n")

print("Evaluation complete! Results saved in performance_comparison.txt.")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Evaluating RLHF model...


Some weights of the model checkpoint at /kaggle/input/saved-models/ppo_fine_tuned_model were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Evaluating: 100%|██████████| 3000/3000 [41:26<00:00,  1.21batch/s]





Evaluating DPO model...


Evaluating: 100%|██████████| 3000/3000 [41:08<00:00,  1.22batch/s]


Performance Comparison:
RLHF BLEU Score: 0.0565
RLHF ROUGE Scores: {'rouge-1': 0.15713403196932169, 'rouge-2': 0.030624756464551525, 'rouge-l': 0.10300433740379256}
DPO BLEU Score: 0.0559
DPO ROUGE Scores: {'rouge-1': 0.18935719756432268, 'rouge-2': 0.030685849523024365, 'rouge-l': 0.11969934062941627}
Evaluation complete! Results saved in performance_comparison.txt.


In [12]:
import os

# Define original and new file names
original_file = "performance_comparison.txt"
new_file = "evaluation_results.txt"

# Rename the file
os.rename(original_file, new_file)

# Confirm the rename
print(f"File renamed to: {new_file}")
print("Files in /kaggle/working/:", os.listdir("/kaggle/working/"))


File renamed to: evaluation_results.txt
Files in /kaggle/working/: ['.virtual_documents', 'evaluation_results.txt', 'performance_comparison.zip', 'performance_comparison_txt.zip', 'performance_comparison.txt.zip']


In [13]:
import shutil
import os

file_name = "evaluation_results.txt"
zip_name = "evaluation_results"

# Create a zip archive containing the text file
shutil.make_archive(zip_name, 'zip', root_dir=".", base_dir=file_name)

# Move the zip file to /kaggle/working/
shutil.move(f"{zip_name}.zip", f"/kaggle/working/{zip_name}.zip")

# Confirm the file exists
print("Files in /kaggle/working/:", os.listdir("/kaggle/working/"))

Files in /kaggle/working/: ['.virtual_documents', 'evaluation_results.txt', 'performance_comparison.zip', 'performance_comparison_txt.zip', 'evaluation_results.zip', 'performance_comparison.txt.zip']


In [14]:
%cd /kaggle/working

from IPython.display import FileLink
FileLink("evaluation_results.zip")

/kaggle/working


/kaggle/working/evaluation_results.zip

In [1]:
import torch

roll_no = "20CS30069"  # Replace with your roll number
src_path = "/kaggle/input/more-uploads/reward_model.pth"
dst_path = f"Assignment1_{roll_no}_reward_model.pt"

# Load the model
model_state = torch.load(src_path, map_location="cpu")  # Load on CPU for safety

# Save with new filename
torch.save(model_state, dst_path)

print(f"Model saved as: {dst_path}")


<ipython-input-1-4aded81c760e>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_state = torch.load(src_path, map_location="cpu")  # Load on CPU for safety


Model saved as: Assignment1_20CS30069_reward_model.pt


In [3]:
%cd /kaggle/working

from IPython.display import FileLink
FileLink("Assignment1_20CS30069_reward_model.pt")

/kaggle/working


/kaggle/working/Assignment1_20CS30069_reward_model.pt

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer

roll_no = "20CS30069"  # Replace with your roll number
save_path = "/kaggle/input/saved-models/ppo_fine_tuned_model"
output_path = f"Assignment1_{roll_no}_rlhf_trained.pt"

# Load the fine-tuned model and tokenizer
model = AutoModel.from_pretrained(save_path)
tokenizer = AutoTokenizer.from_pretrained(save_path)

# Load optimizer and scheduler states (optional)
optimizer_state = torch.load(f"{save_path}/optimizer.pth", map_location="cpu")
scheduler_state = torch.load(f"{save_path}/scheduler.pth", map_location="cpu")

# Bundle everything into a dictionary
save_dict = {
    "model_state_dict": model.state_dict(),
    "tokenizer": tokenizer,  # Saving tokenizer as reference (can reload from the same checkpoint)
    "optimizer_state_dict": optimizer_state,
    "scheduler_state_dict": scheduler_state
}

# Save as .pt file
torch.save(save_dict, output_path)

print(f"RLHF-trained model saved as: {output_path}")


<ipython-input-4-f0e94517fc27>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  optimizer_state = torch.load(f"{save_path}/optimizer.pth", map_location="cpu")
<ipython-inpu

RLHF-trained model saved as: Assignment1_20CS30069_rlhf_trained.pt


In [5]:
%cd /kaggle/working

from IPython.display import FileLink
FileLink("Assignment1_20CS30069_rlhf_trained.pt")

/kaggle/working


/kaggle/working/Assignment1_20CS30069_rlhf_trained.pt

In [8]:
import torch
from transformers import AutoModel, AutoTokenizer

# Define roll number
roll_no = "20CS30069"  # Replace with your actual roll number

# Define paths
model_path = "/kaggle/input/saved-models/gpt2-medium-dpo"
output_file = f"Assignment1_{roll_no}_dpo_trained.pt"

# Load model and tokenizer
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Prepare dictionary for saving
save_dict = {
    "model_state_dict": model.state_dict(),
    "tokenizer": tokenizer  # Tokenizer can be reloaded from the same checkpoint
}

# Save as .pt file
torch.save(save_dict, output_file)

print(f"Fine-tuned DPO model saved as: {output_file}")


Fine-tuned DPO model saved as: Assignment1_20CS30069_dpo_trained.pt


In [9]:
%cd /kaggle/working

from IPython.display import FileLink
FileLink("Assignment1_20CS30069_dpo_trained.pt")

/kaggle/working


/kaggle/working/Assignment1_20CS30069_dpo_trained.pt

In [10]:
import sys

print("Python version:", sys.version)
print("Python version info:", sys.version_info)

Python version: 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
Python version info: sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)
